In [221]:
import os
import ast
import json
import importlib 
import itertools
import collections
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr


import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw


from IPython import display

importlib.reload(bestfits)
importlib.reload(setup_bestfit)
importlib.reload(setup_overlap)
importlib.reload(pipe_lblnew)
importlib.reload(setup_cliradlw)
importlib.reload(pipe_cliradlw)

<module 'climatools.cliradlw.pipeline' from '/chia_cluster/home/jackyu/climatools/climatools/cliradlw/pipeline.py'>

In [129]:
pd.set_option('max_colwidth', -1)

In [130]:
def bandmap_lblnew2clirad():
    '''
    Band map: lblnew ---> clirad
    '''
    d = {'1': '1',
                     '2': '2',
                     '3a': '3',
                     '3b': '4',
                     '3c': '5',
                     '4': '6', 
                     '5': '7',
                     '6': '8',
                     '7': '9', 
                     '8': '10',
                     '9': '11'}
    return d



def bandmap_clirad2lblnew():
    '''
    Band map: clirad ---> lblnew
    '''
    d_lblnew2clirad = bandmap_lblnew2clirad()
    return {v: k for k, v in d_lblnew2clirad.items()}



In [131]:
def get_hrefs_clirad(param=None):
    atmpros = ('mls', 'saw', 'trp')
        
    hrefs = []
    for atmpro in atmpros:
        p = param.copy()
        p['atmpro'] = atmpro
        url = pipeline_clirad.nbviewer_url(param=p, setup=setup_clirad)
        
        s = '''<a href="{}">{}</a>'''
        s = s.format(url, atmpro)
        hrefs.append(s)
    s_hrefs = ' '.join(hrefs)
    return s_hrefs



def tb_hrefs_clirad():
    df = pd.DataFrame()
    
    for param in setup_clirad.test_cases():
        
        hrefs = get_hrefs_clirad(param=param)
        
        bands = [str(b) for b in param['band']]
        
        df.loc['+'.join(param['molecule']), 
               '+'.join(bands)] = hrefs
        
    df = df.fillna('')
    
    bmap = bandmap_clirad2lblnew()
    lblnew_bands = [bmap[b] for b in df.columns]
    columns = pd.MultiIndex.from_arrays([lblnew_bands, df.columns],
                                        names=['lblnew', 'clirad'])
    df.columns = columns
    

    return df
        


In [217]:
'''
Get the clirad-lw and lblnew `param`s for all spectral bands.  
These are returned by functions `clirad_params_atm` and
`lblnew_params_atm`, respectively.
'''

def molecules_byband_atm():
    return {1: {'h2o': 'atmpro'},
            2: {'h2o': 'atmpro'}, 
            3: {'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 3.2e-07}}


def clirad_params_atm():
    d = {}
    for band, molecule in molecules_byband_atm().items():
        for param in setup_clirad.test_cases():
            if [band] == param['band'] and molecule == param['molecule']:
                d[band] = param
                break                
    return d


def analysis_dirs_atm():
    params = clirad_params_atm()
    return {band: pipe_cliradlw.get_analysis_dir(param=param,
                                                 setup=setup_clirad) 
            for band, param in params.items()}


def lblnew_params_atm():
    dirs = analysis_dirs_atm()
    
    d = {}
    for band, dirname in dirs.items():
        with open(os.path.join(dirname, 'param.py'), 
                  mode='r', encoding='utf-8') as f:
            _, l = f.readlines()
        
        s = l.split('=')[1].strip()
        d[band] = ast.literal_eval(s)
        
    return d

    


In [218]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [219]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict

In [276]:
def crd_data_atm(params_atm):
    '''
    Parameters
    ----------
    
    params_atm: dict
                Parameter dictionary for each band.
                
    d: dict
       'flux': xr.Dataset. [pressure, band]
               Fluxes.
       'cool': xr.Dataset. [pressure, band]
               Cooling rate.
    
    '''
    
    results_atm = {band: load_lblnew_data(param) 
                   for band, param in params_atm.items()}
    
    bands = [band for band, _ in params_atm.items()]
    fluxs = [d['ds_flux_crd'] for _, d in results_atm.items()]
    cools = [d['ds_cool_crd'] for _, d in results_atm.items()]
    
    d = {}
    d['flux'] = xr.concat(fluxs, dim=bands).rename({'concat_dim': 'band'})
    d['cool'] = xr.concat(cools, dim=bands).rename({'concat_dim': 'band'})
    return d        
        
        

def clirad_data_atm(params_atm):
    
    dirnames = [pipe_cliradlw.get_fortran_dir(param,
                                              setup=setup_cliradlw)
                for _, param in params_atm.items()]
    
    fpaths_flux = [os.path.join(n, 'output_flux.dat') for n in dirnames]
    fpaths_cool = [os.path.join(n, 'output_coolr.dat') for n in dirnames]
    
    bands = [band for band, _ in params_atm.items()]
    fluxs = [load_output_file(p) for p in fpaths_flux]    
    cools = [load_output_file(p) for p in fpaths_cool]
    
    d = {}
    d['flux'] = sum(fluxs).sel(band=bands)
    d['cool'] = sum(cools).sel(band=bands)
    return d


    
d_crd = crd_data_atm(lblnew_params_atm())
d_clirad = clirad_data_atm(clirad_params_atm())


In [277]:
d_crd

{'cool': <xarray.Dataset>
 Dimensions:   (band: 3, pressure: 75)
 Coordinates:
   * pressure  (pressure) float64 0.0003122 0.0007501 0.001052 0.001476 ...
     layer     (pressure) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
   * band      (band) int64 1 2 3
 Data variables:
     coolrg    (band, pressure) float64 6.538 5.055 4.529 3.992 3.463 2.965 ...,
 'flux': <xarray.Dataset>
 Dimensions:   (band: 3, pressure: 76)
 Coordinates:
   * pressure  (pressure) float64 0.0 0.0006244 0.0008759 0.001229 0.001723 ...
     level     (pressure) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
   * band      (band) int64 1 2 3
 Data variables:
     flug      (band, pressure) float64 -34.26 -34.26 -34.26 -34.26 -34.26 ...
     fldg      (band, pressure) float64 0.0 0.0005011 0.000659 0.0008575 ...
     fnetg     (band, pressure) float64 -34.26 -34.26 -34.26 -34.26 -34.26 ...}

In [278]:
d_clirad

{'cool': <xarray.Dataset>
 Dimensions:   (band: 3, pressure: 75)
 Coordinates:
   * pressure  (pressure) float64 0.0003122 0.0007501 0.001052 0.001476 ...
     layer     (pressure) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
   * band      (band) int64 1 2 3
 Data variables:
     coolrg    (band, pressure) float64 3.93 3.665 3.533 3.346 3.094 2.775 ...,
 'flux': <xarray.Dataset>
 Dimensions:   (band: 3, pressure: 76)
 Coordinates:
     level     (pressure) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
   * pressure  (pressure) float64 1e-08 0.0006244 0.0008759 0.001229 0.001723 ...
   * band      (band) int64 1 2 3
 Data variables:
     flug      (band, pressure) float64 -34.07 -34.07 -34.07 -34.07 -34.07 ...
     fldg      (band, pressure) float64 5.671e-07 0.0003078 0.0004237 ...
     fnetg     (band, pressure) float64 -34.07 -34.07 -34.07 -34.07 -34.07 ...}

In [244]:
bands = [band for band, _ in crd_flux.items()]
dss = [ds for _, ds in crd_flux.items()]

ds = xr.concat(dss, dim=bands)
ds = ds.rename({'concat_dim': 'band'})

In [249]:
ds['flug']

<xarray.DataArray 'flug' (band: 3, pressure: 76)>
array([[-34.263992, -34.26401 , -34.264017, ..., -50.704381, -50.95136 ,
        -51.073429],
       [-60.366689, -60.366695, -60.366697, ..., -81.57426 , -82.074454,
        -82.315795],
       [-24.866066, -24.866085, -24.866092, ..., -35.930664, -36.139676,
        -36.235408]])
Coordinates:
  * pressure  (pressure) float64 0.0 0.0006244 0.0008759 0.001229 0.001723 ...
    level     (pressure) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * band      (band) int64 1 2 3

In [133]:
def script():
    df = tb_hrefs_clirad()
    
    s_html = df.to_html(escape=False)
    display.display(display.HTML(s_html))

In [134]:
script()

lblnew,1,2,3a,3b,3c,4,5,6,7,8,9
clirad,1,2,3,4,5,6,7,8,9,10,11
h2o,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp
co2,,,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,,,,mls saw trp
o3,,,,,,,mls saw trp,,,,mls saw trp
n2o,,,mls saw trp,,,,,,mls saw trp,,
ch4,,,,,,,,,mls saw trp,,
co2+h2o+n2o,,,mls saw trp,,,,,,,,
co2+h2o,,,,mls saw trp,mls saw trp,mls saw trp,,,,,mls saw trp
co2+h2o+o3,,,,,,,mls saw trp,,,,
h2o+o3,,,,,,,mls saw trp,,,,


In [135]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')